In [1]:
% env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
% matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

### 讀入 MNIST

In [3]:
from keras.datasets import mnist

Using TensorFlow backend.


In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 整理輸入格式

In [5]:
# CNN 要的是 (28, 28, 1)

In [6]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

### 整理輸出格式

In [7]:
# 改成 i-hot encoding

In [8]:
from keras.utils import np_utils

In [9]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [10]:
x_train = x_train / 255
x_test = x_test / 255

In [11]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import SGD

### 建構神經網路

In [12]:
model = Sequential()

- 第一次 Convolution
    - 16 個 filters
    - 16 * 16
    - 用 ELU 作為 Activation Function

In [13]:
model.add(Conv2D(16, 
                 (16, 16), 
                 padding = 'same', 
                 input_shape = (28, 28, 1)))
model.add(Activation('elu'))

- Max-Pooling

In [14]:
model.add(MaxPool2D(pool_size = (2, 2)))

- 第二次 Convolution
    - 32 個 filters
    - 16 * 16
    - 用 ELU 作為 Activation Function

In [15]:
model.add(Conv2D(32, 
                 (16, 16), 
                 padding = 'same'))
model.add(Activation('elu'))

In [16]:
model.add(MaxPool2D(pool_size = (2, 2)))

- 第三次 Convolution
    - 64 個 filters
    - 32 * 32
    - 用 ReLU 作為 Activation Function

In [17]:
model.add(Conv2D(64, 
                 (32, 32), 
                 padding = 'same'))
model.add(Activation('relu'))

In [18]:
model.add(MaxPool2D(pool_size = (2, 2)))

- 送進一般的神經網路，記得拉平！

In [19]:
model.add(Flatten())
model.add(Dense(16))
model.add(Activation('relu'))

- 輸出

In [20]:
model.add(Dense(10))

In [21]:
model.add(Activation('softmax'))

### 組裝：要做 compile 才正式把神經網路建好

In [22]:
model.compile(loss = 'mse',
              optimizer = SGD(lr = 0.087),
              metrics = ['accuracy'])

### 檢視神經網路

In [23]:
model.summary()

# 104 = 5 * 5 * 4 + 4 (bias)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        4112      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        131104    
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          2097216   
__________

### 訓練

In [24]:
model.fit(x_train, y_train, 
          batch_size = 100,
          epochs = 16)

Epoch 1/16
60000/60000 [==============================] - 1718s 29ms/step - loss: 0.0890 - acc: 0.2707
Epoch 2/16
60000/60000 [==============================] - 1815s 30ms/step - loss: 0.0606 - acc: 0.6087
Epoch 3/16
60000/60000 [==============================] - 1599s 27ms/step - loss: 0.0237 - acc: 0.8420
Epoch 4/16
60000/60000 [==============================] - 1889s 31ms/step - loss: 0.0150 - acc: 0.9047
Epoch 5/16
60000/60000 [==============================] - 1716s 29ms/step - loss: 0.0111 - acc: 0.9290
Epoch 6/16
60000/60000 [==============================] - 1727s 29ms/step - loss: 0.0091 - acc: 0.9419
Epoch 7/16
60000/60000 [==============================] - 1668s 28ms/step - loss: 0.0076 - acc: 0.9516
Epoch 8/16
60000/60000 [==============================] - 1796s 30ms/step - loss: 0.0067 - acc: 0.9574
Epoch 9/16
60000/60000 [==============================] - 2098s 35ms/step - loss: 0.0060 - acc: 0.9623
Epoch 10/16
60000/60000 [==============================] - 2219s 37ms/ste

### 訓練結果：準確率為 97.7%，比上課示範之模型 92.9% 進步不少。